# Webexport
Create Popup-GeoJSON

In [8]:
import geopandas as gpd
import pandas as pd
import json
import consts
from utils import categorize_time_to_half_hour, categorize_time_to_half_hour_numerical, time_to_time_str, dt_to_time, time_to_time, categorize_time_to_6min_numerical, categorize_time_to_6min

In [9]:
# Load timedata
time_data = json.load(open(consts.PATH_TIMEDATA_HEX, 'r', encoding='utf-8'))

# Load Nuts3
gdf_grid_raw = gpd.read_file(consts.PATH_HEXAGON)

In [12]:
# Serialize Timedata
df_timedata = list(map(lambda x: {
    'nuts_id': x['nuts_id'],

    'sunrise_latest_summer': x['sunrise']['summer']['latest'],
    'sunrise_latest_winter': x['sunrise']['winter']['latest'],
    'sunrise_latest_current': x['sunrise']['current']['latest'],

    'sunrise_earliest_summer': x['sunrise']['summer']['earliest'],
    'sunrise_earliest_winter': x['sunrise']['winter']['earliest'],
    'sunrise_earliest_current': x['sunrise']['current']['earliest'],

    'sunset_earliest_summer': x['sunset']['summer']['earliest'],
    'sunset_earliest_winter': x['sunset']['winter']['earliest'],
    'sunset_earliest_current': x['sunset']['current']['earliest'],

    'sunset_latest_summer': x['sunset']['summer']['latest'],
    'sunset_latest_winter': x['sunset']['winter']['latest'],
    'sunset_latest_current': x['sunset']['current']['latest'],

    'hidepopup': x['hidepopup']
    
    # 'latest_sunrise_summer_day': x['sunrise']['summer']['latest_day'],
    # 'earliest_sunset_summer_day': x['sunset']['summer']['earliest_day'],
}, time_data))

df_timedata = pd.DataFrame(df_timedata)

# Convert strings to time
def convert_time(x):
    df_timedata[x] = df_timedata[x].apply(time_to_time_str)

# for x in [
#     'sunrise_latest_summer', 'sunrise_latest_winter', 'sunrise_latest_current',
#     'sunrise_earliest_summer', 'sunrise_earliest_winter', 'sunrise_earliest_current',
#     'sunset_earliest_summer', 'sunset_earliest_winter', 'sunset_earliest_current',
#     'sunset_latest_summer', 'sunset_latest_winter', 'sunset_latest_current'
# ]:
#     convert_time(x)

# Define how detailed the output should be (half hour? 5min?)
# categorizes_function = categorize_time_to_half_hour
# categorizes_numerical_function = categorize_time_to_half_hour_numerical
categorizes_function = categorize_time_to_6min
categorizes_numerical_function = categorize_time_to_6min_numerical

def calculate_event(event):

    # Current System
    df_timedata[f'{event}_current_numerical'] = df_timedata[f'{event}_current'].apply(categorizes_numerical_function)

    # Everything on summer
    df_timedata[f'{event}_summer_numerical'] = df_timedata[f'{event}_summer'].apply(categorizes_numerical_function)
    
    # Everything on winter
    df_timedata[f'{event}_winter_numerical'] = df_timedata[f'{event}_winter'].apply(categorizes_numerical_function)

# calculate_event('sunrise_earliest')    
# calculate_event('sunset_earliest')    
# calculate_event('sunset_latest')    

# Merge Dataframes
gdf_grid = gdf_grid_raw.merge(df_timedata, left_on='NUTS_ID', right_on='nuts_id')
gdf_grid = gdf_grid.drop(columns=['nuts_id'])

# Drop unwanted Timezones
gdf_grid = gdf_grid[~gdf_grid['timezone'].isin(consts.REMOVE_TIMEZONES)]

## Export GeoJSON

In [14]:
gdf_grid.to_file(consts.PATH_GEOJSON_POPUPDATA)